to do list
1. transition to factor model 
2. make the use of full simulation of the encoder model 
     1. demonstrate the terrible effect of not having a feature model
     2. try the default regular interval update sort of thing
3. add non-linear feature mixing

Questions
1. what would be a realistic sim scenerio in terms of number of trials to go?

In [4]:
#test imports from the riglib feature library alright
#trying to hijack the encoder class. 
#where is it? 
#using relative import, okay? 
import numpy as np
import sys
sys.path.append('..//riglib')

import riglib.bmi.sim_neurons as sim_neurons
from riglib.bmi.state_space_models import StateSpaceEndptVel2D

# initial set up of the state space model

In [7]:
#set up a state space model sort of thing
ssm = StateSpaceEndptVel2D()

# specify a control matrix
N_NEURONS = 20
N_STATES = 7  # 3 positions and 3 velocities and an offset


# build the observation matrix
sim_C = np.zeros((N_NEURONS, N_STATES))
# control x positive directions
sim_C[2, :] = np.array([0, 0, 0, 1, 0, 0, 0])
sim_C[3, :] = np.array([0, 0, 0, -1, 0, 0, 0])
# control z positive directions
sim_C[5, :] = np.array([0, 0, 0, 0, 0, 1, 0])
sim_C[6, :] = np.array([0, 0, 0, 0, 0, -1, 0])



In [8]:
A, B, W = ssm.get_ssm_matrices()
print(A)
print(B)

[[1.  0.  0.  0.1 0.  0.  0. ]
 [0.  1.  0.  0.  0.1 0.  0. ]
 [0.  0.  1.  0.  0.  0.1 0. ]
 [0.  0.  0.  0.8 0.  0.  0. ]
 [0.  0.  0.  0.  0.8 0.  0. ]
 [0.  0.  0.  0.  0.  0.8 0. ]
 [0.  0.  0.  0.  0.  0.  1. ]]
[[  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [100.   0.   0.]
 [  0. 100.   0.]
 [  0.   0. 100.]
 [  0.   0.   0.]]


the top matrix is our state transition matrix. and the velocities are by default decaying by a factor of 0.8 and 10% of velocity is  converted to the state for whatever reason. 

the bottom is the control input to drive the state. 
then we can start to set up the encoder sort of thing.

# initialize the encoder sort of thing

## use a cosine tuned model (no untuned neurons)

this class has this sort of initialization parameters

class GenericCosEnc(object):
    '''
    Simulate neurons where the firing rate is a linear function of covariates and the rate parameter goes through a Poisson
    '''
    def __init__(self, C, ssm, return_ts=False, DT=0.1, call_ds_rate=6):
        '''
        Constructor for GenericCosEnc

        Parameters
        ----------
        C : np.ndarray of shape (N, K)
            N is the number of simulated neurons, K is the number of covariates driving neuronal activity. 
            The product of C and the hidden state vector x should give the intended spike rates in Hz
        ssm : state_space_models.StateSpace instance
            ARG_DESCR
        call_ds_rate : int, optional, default=6
            Calculating DT / call_ds_rate gives the interval between ticks of the main event loop


In [33]:

intended_state = np.asarray([0, 0, 0, 10, 0, 10, 1]).T #transpose into a col vector
#need to seed the pseudo-random generator
np.random.seed(seed= 6)

#the rates are simply poisson distributed counts
rates = np.dot(sim_C, intended_state)
from numpy.random import poisson
rates[rates < 0] = 0 # Floor firing rates at 0 Hz
counts = poisson(rates * 0.1) # DT = 0.1 the time interval
print('rates are')
print(rates)
print('counts are')
print(counts)

rates are
[ 0.  0. 10.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
counts are
[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [34]:
np.random.seed(seed= 6)
cos_tuned_encoder = sim_neurons.GenericCosEnc(sim_C, ssm,
                                             call_ds_rate=1) #for sim, we want this to return us every call, 

#get the encoded firing rates by
#calling the encoder, very convenient
print(cos_tuned_encoder(intended_state))

[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


what this means is that during this short time interval (dT = 0.1), neuron 2 and neuron 5 fired 1 once, corresponding to our set obervation matrix setup

therefore, it looks like this obervation matrix will be our main point of contention, right? this seems to be Andrew Schwartz's model but we are not done yet. 

## use the factor model with both tuned and untuned neurons 

In [35]:
fa_cos_tuned_encoder = sim_neurons.FACosEnc(sim_C, ssm,
                                             call_ds_rate=1) #for sim, we want this to return us every call, 

In [40]:
#explore a bit what parameters have been initialized
print(fa_cos_tuned_encoder.wt_sources) # if it's none, then equally weight the sources 
# w = np.array([1,1,1,1]) / 4
#  counts = np.squeeze(np.array(w[0]*self.priv_unt 
#                             + w[1]*self.priv_tun 
#                             + w[2]*self.shar_unt 
#                             + w[3]*self.shar_tun))

print(fa_cos_tuned_encoder.n_facts)
# Establish number factors for tuned / untuned input sources: 
#self.n_tun_factors = self.n_facts[0] #1
#self.n_unt_factors = self.n_facts[1]


None
[3, 3]
